In [1]:
import numpy as np
from DCP import *

%load_ext autoreload
%autoreload 2


In [ ]:
# FM Forward Matrix
# CM Color Matrix
# AB Anology Balance /Camera White balance camera_whitebalance

In [4]:
parser = DCPParser('E:\摄影Photo软件工具\DCP_Projects\dcp转换工具20200618\Sony ILCE-7RM2 Adobe Standard.xml')
parser.parse_matrices()
CM1 = parser.matrices['ColorMatrix1']
CM2 = parser.matrices['ColorMatrix2']
FM1 = parser.matrices['ForwardMatrix1']
FM2 = parser.matrices['ForwardMatrix2']

CameraWB = np.array([2592.0, 1024.0, 1556.0])
CameraN = 1/CameraWB
# AB = np.diag([1.83203125,1.0       ,3.15234375])
# BA = np.diag(1/np.array([1.83203125,1.0       ,3.15234375]))


In [5]:
# WB Temp calculation
CM = CM2
# CM = interpolateMatrix(2870,6500,3050,CM1,CM2)

XYZ2Camera = CM # AB * CM
Camer2XYZ = np.linalg.inv(CM) # inv(CM)  * inv(AB) 
# XYZ white point = Camer2XYZ * Camera netural point
# Camera netural point is (1/Camera WB) just CameraRGB response
XYZ_wp = Camer2XYZ @ CameraN

x,y,Y = XYZ2xyY(XYZ_wp)
xy2Temp(x,y),xyCoordToTemperature([x,y])


(np.float64(5401.177271224945), np.float64(5403.438174019584))

In [7]:
# Forawrd Matrix Test

print(FM1 @ [1,1,1]) # D50 in XYZ
print(FM2 @ [1,1,1]) # D50 in XYZ

[0.9644 1.     0.8251]
[0.9644 1.     0.8251]


# ISP process

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from ISP import *

# %matplotlib notebook
%matplotlib qt5

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
img,H_raw = read_raw_simple("./0.ARW")

path ='E:\摄影Photo软件工具\DCP_Projects\dcp转换工具20200618\Sony ILCE-7RM2 Adobe Standard.xml'
path = 'E:\摄影Photo软件工具\DCP_Projects\dcp转换工具20200618\Olympus E-M1 Mark III Adobe Standard.xml'

parser = DCPParser(path)
parser.parse_matrices()
CM1 = parser.matrices['ColorMatrix1']
CM2 = parser.matrices['ColorMatrix2']
FM1 = parser.matrices['ForwardMatrix1']
FM2 = parser.matrices['ForwardMatrix2']

# Camera AWB
R_gain,G_gain,B_gain = H_raw.camera_whitebalance[0:3]
cameraWB = np.array([R_gain/G_gain,1,B_gain/G_gain])
cameraNetural = 1/cameraWB # Actuall is the RGB vector represent white

RAW Shape:(5320, 8000)
OB 512 
 white_level 16383 
 MAX16628


In [72]:
# WB Temp calculation
CM = CM2
# CM = interpolateMatrix(2870,6500,3050,CM1,CM2)

XYZ2Camera = CM # AB * CM
Camer2XYZ = np.linalg.inv(CM) # inv(CM)  * inv(AB) 
# XYZ white point = Camer2XYZ * Camera netural point
# Camera netural point is (1/Camera WB) just CameraRGB response
XYZ_wp = Camer2XYZ @ cameraNetural

x,y,Y = XYZ2xyY(XYZ_wp)
xy2Temp(x,y),xyCoordToTemperature([x,y])

(np.float64(4164.80627586312), np.float64(4185.195926227314))

In [29]:
#https://www.russellcottrell.com/photo/matrixCalculator.htm

# D50 XYZ 2 sRGB
XYZ2RGB = np.array(
[[ 3.1342599, -1.6171978, -0.4906852],
 [-0.9787551,  1.9161350,  0.0334462],
 [ 0.0719423, -0.2289582,  1.4052060]])

XYZ2RGB @ [0.96422,	1.00000,	0.82521] #should be [1,1,1]

array([0.99999995, 0.9999999 , 1.00000005])

In [73]:
# Forward Matrix
FM = FM2
D = np.diag(cameraWB)
Camera2XYZD50 = FM @ D

vector, invf = im2vector(img) # [N,3]

wb_img = D @ vector.T # AWBed CameraRGB
XYZ_img = Camera2XYZD50 @ vector.T # [3,N] in D50
sRGB_img = XYZ2RGB @ XYZ_img # linear RGB


In [61]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plotting the first image
axs[0, 0].imshow(img)
axs[0, 0].set_title('Camera RGB (No gamma)')

# Plotting the second image
axs[0, 1].imshow(invf(gamma(wb_img.T)))
axs[0, 1].set_title('AWB Camera RGB')

# Plotting the third image
axs[1, 0].imshow(invf(XYZ_img.T))
axs[1, 0].set_title('XYZ D50')

# Plotting the fourth image
axs[1, 1].imshow(invf(gamma(sRGB_img.T)))
axs[1, 1].set_title('sRGB')

plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..1.015437].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..2.4469810269223062].


In [74]:
# Test Different Colors

color1 = FM1 @ wb_img # [N,3]
color2 = FM2 @ wb_img

RGB1 = gamma(XYZ2RGB @ color1).T # linear RGB > sRGB
RGB2 = gamma(XYZ2RGB @ color2).T

RGBC = gamma(wb_img).T




In [75]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plotting the second image
axs[0, 0].imshow(invf(RGBC))
axs[0, 0].set_title('AWB Camera RGB')

# Plotting the first image
axs[0, 1].imshow(invf(RGB2))
axs[0, 1].set_title('FM2 sRGB')

# Plotting the third image
axs[1, 0].imshow(invf(RGB1))
axs[1, 0].set_title('FM1 sRGB')

Text(0.5, 1.0, 'FM1 sRGB')

In [78]:
# PLay with FM

XYZ2RGB @ FM1

array([[ 1.11961491,  0.03195207, -0.15131139],
       [-0.11214449,  1.12796062, -0.01589487],
       [ 0.09576729, -0.15242567,  1.05665014]])

In [53]:
plt.figure()
plt.imshow(invf(gamma(wb_img.T)))
plt.title("AWB Camera RGB")
print(wb_img.max(),wb_img.min())

In [77]:
XYZ2RGB @ FM

array([[ 0.9035127 ,  0.04414464,  0.05296076],
       [-0.01299502,  1.05110494, -0.03828987],
       [ 0.12542724, -0.1427752 ,  1.01720638]])